# Programa para deducir la matriz de rigidez de un elemento de viga de Timoshenko-Ehrenfest a partir de la solución de la ecuacion diferencial

In [1]:
import sympy as sp
from sympy import integrate, simplify

# Para imprimir bonito
sp.init_printing()

from IPython.display import Math                 
def imprimir (texto, variable, large=False):
    if large:
        return Math(texto + r'\Large{' + rf'{sp.latex(variable)}' + '}')
    else:
        return Math(texto +  rf'{sp.latex(variable)}')

In [2]:
# %%Definción de variables
x, L, EI, beta = sp.symbols('x L EI beta')

# Se definen las constantes de integración
C1, C2, C3, C4 = sp.symbols('C_1, C_2, C_3, C_4')

# Se define beta como beta = (12 * EI)/(L**2 * GAast)
GAast = (12 * EI)/(L**2 * beta)

# Se define la carga: hace la ecuación diferencial homogénea
q = 0

## Cálculo de la matriz de rigidez $\boldsymbol{K}_{TE}$

### Método 1

In [3]:
# Se plantea la ecuación diferencial
V = integrate(q, x)           + C1  # Ecuaciones calculadas con constantes.
M = integrate(V, x)           + C2
t = (integrate(M, x)          + C3)/EI
w = integrate(t - V/GAast, x) + C4

# Se calcula la matrix de rigidez
K_TE = sp.zeros(4)
for i in range(4):
    sol = sp.solve([w.subs(x, 0) - int((i == 0)),  # Condiciones de frontera
                    t.subs(x, 0) - int((i == 1)),
                    w.subs(x, L) - int((i == 2)),
                    t.subs(x, L) - int((i == 3))],
                [C1, C2, C3, C4])

    constantes = [(C1, sol[C1]), (C2, sol[C2]), (C3, sol[C3]), (C4, sol[C4])]

    K_TE[:, i] = [+ (V.subs(constantes)).subs(x, 0),   # Y1  se evaluan las
                  - (M.subs(constantes)).subs(x, 0),   # M1  reacciones verticales
                  - (V.subs(constantes)).subs(x, L),   # Y2  y los momentos en los
                  + (M.subs(constantes)).subs(x, L)]   # M2  apoyos

# %%Se imprime la solución
tmp = EI/((1 + beta)*L**3)
imprimir(r'K_{TE} = \frac{E I}{(1 + \beta) L^3}', simplify(K_TE/tmp))

<IPython.core.display.Math object>

### Método 2

In [4]:
V = sp.Function('V')(x)
M = sp.Function('M')(x)
t = sp.Function('theta')(x)
w = sp.Function('w')(x)
    
# Se calcula la matrix de rigidez
K_TE = sp.zeros(4)
for i in range(4):
    sol = sp.dsolve(eq=[  sp.diff(V,x) - 0,
                          sp.diff(M,x) - V,
                          sp.diff(t,x) - M/EI,
                          sp.diff(w,x) - t + V/GAast   ],
                    ics={ w.subs(x, 0): int((i == 0)),
                          t.subs(x, 0): int((i == 1)),
                          w.subs(x, L): int((i == 2)),
                          t.subs(x, L): int((i == 3))  })
    VV = sol[0].rhs
    MM = sol[1].rhs
    
    K_TE[:, i] = [+ VV.subs(x, 0),   # Y1  se evaluan las
                  - MM.subs(x, 0),   # M1  reacciones verticales
                  - VV.subs(x, L),   # Y2  y los momentos en los
                  + MM.subs(x, L)]   # M2  apoyos

# %%Se imprime la solución
tmp = EI/((1 + beta)*L**3)
imprimir(r'\boldsymbol{K}_{TE} = \frac{E I}{(1 + \beta) L^3}', simplify(K_TE/tmp))

<IPython.core.display.Math object>

## Cálculo de la matriz de rigidez de la teoría de Euler-Bernoulli $\boldsymbol{K}_{EB}$
Observe que cuando $GA^* \to \infty$ (o alternativamente $\beta \to 0$), la matriz de rigidez $\boldsymbol{K}_{TE}$ se vuelve la misma matriz de rigidez $\boldsymbol{K}_{EB}$ de la teoría de Euler-Bernoulli:

In [5]:
K_EB = K_TE.limit(beta, 0)

# %%Se imprime la solución
tmp = EI/(L**3)
imprimir(r'\boldsymbol{K}_{EB} = \frac{E I}{L^3}', simplify(K_EB/tmp))

<IPython.core.display.Math object>

## Cálculo del vector de fuerzas nodales equivalentes

In [6]:
w1, w2 = sp.symbols('w_1 w_2')
q = (w2 - w1)*x/L + w1

# Se plantea la ecuación diferencial
V = integrate(q, x)           + C1  # Ecuaciones calculadas con constantes.
M = integrate(V, x)           + C2
t = (integrate(M, x)          + C3)/EI
w = integrate(t - V/GAast, x) + C4

# Se calcula la matrix de rigidez
sol = sp.solve([w.subs(x, 0),  # Condiciones de frontera
                w.subs(x, L),
                t.subs(x, 0),
                t.subs(x, L)],
                [C1, C2, C3, C4])

constantes = [(C1, sol[C1]), (C2, sol[C2]), (C3, sol[C3]), (C4, sol[C4])]

f_TE = sp.Matrix([-(V.subs(constantes)).subs(x, 0),   # Yi  se evaluan las reacciones verticales y los
                  +(M.subs(constantes)).subs(x, 0),   # Mi  momentos en los apoyos y se les multiplica
                  +(V.subs(constantes)).subs(x, L),   # Yj  por -1 para estimar la fuerza nodal 
                  -(M.subs(constantes)).subs(x, L)])  # Mj  equivalente

In [12]:
imprimir(r'\boldsymbol{f}_{TE} = ', simplify(f_TE))
#sp.pprint(simplify(f_TE))

<IPython.core.display.Math object>

In [8]:
f_EB = f_TE.limit(beta,0)
imprimir(r'\boldsymbol{f}_{EB} = ', simplify(f_EB))

<IPython.core.display.Math object>